In [1]:
import os

import pandas as pd

import cv2
import numpy as np

from PIL import Image

In [2]:
data_dir = 'media/for_kills_only/'
out_dir = 'media/kills_only/'

## Crop and Relocate Images

In [3]:
existing = pd.read_csv('numbers.csv')
n_existing = len(existing)

df_to_be_converted = pd.read_csv(f'{data_dir}sample_records.csv')
preds_n_kills = df_to_be_converted.n_kills.values
reference_files_to_be_converted = df_to_be_converted.reference_file.values
n_to_be_converted = len([f for f in os.listdir(data_dir) if '.jpg' in f])

In [4]:
_ = 0
kills_files = []

In [5]:
for f in range(n_existing, n_existing + n_to_be_converted):
    image = Image.open(reference_files_to_be_converted[_])
    _ += 1
    
    if image.size != (28, 28):
        image = image.crop((139, 0, 167, 28))
        
    if f < 10:
        fn = f'loop_0000{f}'
    elif f < 100:
        fn = f'loop_000{f}'
    elif f < 1000:
        fn = f'loop_00{f}'
    elif f < 10000:
        fn = f'loop_0{f}'
    else:
        raise Exception(f'\nf > 10,000\nf = {f}\n')
        
    fp = f'{out_dir}{fn}.jpg'
    image.save(fp)
    kills_files.append(fp)

In [6]:
if n_to_be_converted > 0:
    for file in reference_files_to_be_converted:
        os.remove(file)

## Label Cropped Images
Any of them (may have some left over from previous sessions, they will be first in line).

In [7]:
# existing = pd.read_csv('numbers.csv')

seen_files = [i for i in existing.file_path.values]
n_seen_files = len(seen_files)
numbers = [i for i in existing.numbers.values]

unseen_files = [f'{out_dir}{f}' for f in os.listdir(out_dir) if f'{out_dir}{f}' not in seen_files and '.jpg' in f]
n_unseen_files = len(unseen_files)

n_correct_preds = 0

In [8]:
if n_to_be_converted != n_unseen_files:
    print(f'Note: {n_unseen_files - n_to_be_converted} preexisting unseen')

print(f'# seen: {n_seen_files}\n# unseen: {n_unseen_files}')

if n_to_be_converted == n_unseen_files:
    display_preds_n_kills = True

# seen: 2114
# unseen: 0


In [9]:
for i in range(n_unseen_files):
    f = unseen_files[i]
    if f in seen_files:
        im = Image.open(f)
        im = im.resize((im.size[0]*8, im.size[1]*8))
        im = np.array(im)
        cv2.imshow('kill_digits', im)
        key = cv2.waitKey(1)
    else:
        if int(f[-9:-4]) % 11 == 0:
            print(f)

        im = Image.open(f)
        im = im.resize((im.size[0]*8, im.size[1]*8))
        im = np.array(im)
        cv2.imshow('kill_digits', im)
        key = cv2.waitKey(500)
        if display_preds_n_kills:
            digits = preds_n_kills[i]
            q = input(f'The digits in this image are {digits}? ')
            if (q != '') and (q.lower() != 'y') and (q.lower() != 'yes'):
                try:
                    digits = int(q)
                except:
                    digits = input('What digits are in this image? ')
            else:
                n_correct_preds += 1
        else:
            digits = input('What digits are in this image? ')
        numbers.append(digits)
        seen_files.append(f)


if n_unseen_files > 0:
    print(f'# correct: {n_correct_preds}\n# possible {n_unseen_files}\n% correct: {n_correct_preds/n_unseen_files*100}')

## Record Input Lables

In [10]:
df = pd.DataFrame()

df['numbers'] = numbers
df['file_path'] = seen_files

df

,numbers,file_path
0,0,media/kills_only/loop_00000.jpg
1,n,media/kills_only/loop_00001.jpg
2,2,media/kills_only/loop_00002.jpg
3,2,media/kills_only/loop_00003.jpg
4,2,media/kills_only/loop_00004.jpg
...,...,...
2109,n,media/kills_only/loop_02109.jpg
2110,n,media/kills_only/loop_02110.jpg
2111,n,media/kills_only/loop_02111.jpg
2112,n,media/kills_only/loop_02112.jpg


In [11]:
df.to_csv('numbers.csv', index=False)

#### See New 

In [12]:
df = pd.read_csv('numbers.csv')

df

,numbers,file_path
0,0,media/kills_only/loop_00000.jpg
1,n,media/kills_only/loop_00001.jpg
2,2,media/kills_only/loop_00002.jpg
3,2,media/kills_only/loop_00003.jpg
4,2,media/kills_only/loop_00004.jpg
...,...,...
2109,n,media/kills_only/loop_02109.jpg
2110,n,media/kills_only/loop_02110.jpg
2111,n,media/kills_only/loop_02111.jpg
2112,n,media/kills_only/loop_02112.jpg


In [13]:
df = df.loc[df.numbers != 'n']
df = df.loc[df.numbers != 'b']
df = df.loc[df.numbers != 'e']

df.numbers.value_counts()

5     250
10    148
12    141
1     136
6     130
13    113
11    107
0      95
2      85
9      82
14     67
8      62
16     61
7      60
15     59
19     50
4      37
3      34
20     28
17     27
22     26
18     25
21     14
25      9
23      7
26      5
27      4
24      3
Name: numbers, dtype: int64